### Agent Workflow

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from langchain.chat_models import init_chat_model

model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=.5)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=.5)
model_gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=.5)

In [ ]:
professional_cold_email_instruction = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

witty_cold_email_instruction = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

precise_cold_email_instruction = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [9]:
##once the best email gets selected, the concluding step would be the agent sends out the email via SendGrid or MailGun platform.
## For now this will be a simple mock tool

from langchain.tools import tool
@tool
def send_email(message: str) -> str:
    """
    Send an email with the provided message content.

    This tool is used to deliver an email message to a predefined recipient
    or mailing configuration. It should be called when the agent needs to
    send information, confirmation, or follow-up details via email.

    When to use:
    - The user explicitly asks to send an email
    - The workflow requires notifying a user via email
    - The agent needs to send confirmation or summary information

    Args:
        message (str): The email body text to be sent.

    Returns:
        str: A confirmation message indicating whether the email
        was sent successfully.

    Output rules:
        - Do NOT explain the tool call
        - Do NOT add conversational text
        - Call this tool directly with extracted values only

    Examples:
        >>> send_email("Your account has been created successfully.")
        'Email message was sent out successfully.'
    """
    return "Email message was sent out successfully."



In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import ModelCallLimitMiddleware

professional_cold_email_agent = create_agent(
    model=model_llama_groq,
    system_prompt=professional_cold_email_instruction,
    middleware=[ModelCallLimitMiddleware(thread_limit=4, run_limit=2, exit_behavior='error')]
)

witty_cold_email_agent = create_agent(
    model=model_llama_groq,
    system_prompt=witty_cold_email_instruction,
    middleware=[ModelCallLimitMiddleware(thread_limit=4, run_limit=2, exit_behavior='error')]
)

precise_cold_email_agent = create_agent(
    model=model_llama_groq,
    system_prompt=precise_cold_email_instruction,
    middleware=[ModelCallLimitMiddleware(thread_limit=4, run_limit=2, exit_behavior='error')]
)

### Create functions/tools to call the subagents

In [ ]:
from langchain.tools import tool
from langchain.messages import HumanMessage

input = {"messages":HumanMessage("Draft a cold email")}

@tool
def profession_and_serious_cold_email_agent()->str:
    """Call this method when you need professional and serious cold email"""
    response = professional_cold_email_agent.invoke(input)
    return response['messages'][-1].content


@tool
def witty_and_engaging_cold_email_agent()->str:
    """Call this method when you need witty and engaging cold email"""
    response = witty_cold_email_agent.invoke(input)
    return response['messages'][-1].content


@tool
def precise_and_tothepoint_cold_email_agent()->str:
    """Call this method when you need concise and to the point cold email"""
    response = precise_cold_email_agent.invoke(input)
    return response['messages'][-1].content



In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
system_prompt="You are sales manager working for ComplAI. You use the tools given to you to generate cold sales emails.\
    You never generate sales emails yourself. You always use the tools. \
    You try all the 3 sales agent tools once before choosing the best one ( The best email is the one which you think the customer will most likely respond to ).\
    The sales agents tools available to you are :  \
    1. profession_and_serious_cold_email_agent \
    2. witty_and_engaging_cold_email_agent \
    3. precise_and_tothepoint_cold_email_agent . \
    You will pick the single best email and use the send_email tool to send the best email ( and only the best email) to the user."

agent = create_agent(
    model=model_gemini_flash,
    tools=[profession_and_serious_cold_email_agent, witty_and_engaging_cold_email_agent, precise_and_tothepoint_cold_email_agent,send_email],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [7]:
from langchain.messages import HumanMessage
from uuid import uuid4

config = {"configurable":{"thread_id": uuid4()}}

response=agent.invoke({"messages":HumanMessage("Draft a cold email for Soc2 compliance")}, config=config)

In [8]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Draft a cold email for Soc2 compliance
================================== Ai Message ==================================
Tool Calls:
  profession_and_serious_cold_email_agent (485c3997-9a40-4c75-93b1-02ef170a509e)
 Call ID: 485c3997-9a40-4c75-93b1-02ef170a509e
  Args:
  witty_and_engaging_cold_email_agent (619e0448-f6a8-4427-baad-1d436e5e47ce)
 Call ID: 619e0448-f6a8-4427-baad-1d436e5e47ce
  Args:
  precise_and_tothepoint_cold_email_agent (62ce55c3-0e99-4abd-818f-089ebd6b0849)
 Call ID: 62ce55c3-0e99-4abd-818f-089ebd6b0849
  Args:
================================= Tool Message =================================
Name: profession_and_serious_cold_email_agent

Subject: Ensure Your Organization's Cybersecurity and Compliance Standards

Dear [Recipient's Name],

I came across [Company Name] and was impressed by your commitment to providing top-notch [industry/field] solutions. As a sales agent for ComplAI, I'd l